In [1]:
using GLPK
using Random
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface

MathOptInterface

## Loading Data

In [2]:
totaltime, totallines = open("T1/R2T1.txt") do f
    global lines = readlines(f);
end

450-element Vector{String}:
 "matrix,row,col"
 "S,72,95"
 "I,J,V"
 "8.0,1.0,-1.0"
 "10.0,1.0,1.0"
 "21.0,1.0,-1.0"
 "43.0,1.0,1.0"
 "50.0,1.0,-1.0"
 "51.0,1.0,1.0"
 "8.0,2.0,1.0"
 "9.0,2.0,-1.0"
 "6.0,3.0,-1.0"
 "12.0,3.0,1.0"
 ⋮
 "68.0,-0.5196655879984552"
 "75.0,-1.679092922126729"
 "76.0,-9.871766517721002e-18"
 "81.0,-0.3156306692199074"
 "82.0,-1.1541689385811602"
 "83.0,-0.9060345343182166"
 "86.0,-0.8810837748807233"
 "89.0,-1.2796727037563316"
 "91.0,-1.5079118796617184"
 "92.0,0.05468668371062502"
 "94.0,-0.6437641587296247"
 "95.0,-0.8732197170766789"

In [3]:
function is_info_line(a_line)
    a_line_info = split(a_line, ",");
    line_indicator = a_line_info[1];
    if cmp(line_indicator, "matrix") == 0 || cmp(line_indicator, "vector") == 0
        return true
    end
    return false
end

is_info_line (generic function with 1 method)

In [4]:
function get_info_end_line(lines_info, t_start, t_end_file)
    current_line = lines_info[t_start];
    t = t_start;
    for t in t_start:t_end_file
        current_line = lines_info[t];
        if is_info_line(current_line)
            return t-1
        end
    end
    return t_end_file
end

get_info_end_line (generic function with 1 method)

In [5]:
function fill_matrix(A, n_rows, n_cols, values_vector)
    for info_line in values_vector
        line_info = split(info_line, ",");
        i_v = floor(Int64, parse(Float64, line_info[1]));
        j_v = floor(Int64, parse(Float64, line_info[2]));
        val = parse(Float64, line_info[3]);
        A[i_v, j_v] = val;
    end
end

fill_matrix (generic function with 1 method)

In [6]:
function fill_vector(vec, n_elements, values_vector)
    for info_line in values_vector
        line_info = split(info_line, ",");
        i_v = floor(Int64, parse(Float64, line_info[1]));
        val = parse(Float64, line_info[2]);
        vec[i_v] = val;
    end
end

fill_vector (generic function with 1 method)

In [7]:
function read_block(t_start)
    info_line = lines[t_start];
    info = split(info_line, ",");
    t = t_start + 1;
    
    if cmp(info[1],"matrix") == 0
        matrix_info_line = lines[t];
        matrix_info = split(matrix_info_line, ",");
        matrix_name = matrix_info[1];
        num_rows = parse(Int64, matrix_info[2]);
        num_cols = parse(Int64, matrix_info[3]);
        if cmp(matrix_name, "S") == 0
            global m = num_rows;
            global n = num_cols;
            global S = zeros(m, n);
        else
            @show "Something is not right here! - id:1"
        end
        t = t+1;
        IJV_line = lines[t];
        if cmp(IJV_line, "I,J,V") != 0
            @show "Something is not right here! - id:2"
        end
        t = t+1;
        t_last = get_info_end_line(lines, t, file_num_lines);
        fill_matrix(S, m, n, lines[t:t_last]);
        t = t_last+1;
        
    elseif cmp(info[1],"vector") == 0
        vector_info_line = lines[t];
        vector_info = split(vector_info_line, ",");
        vector_name = vector_info[1];
        num_elements = parse(Int64, vector_info[2]);
        if parse(Int64, vector_info[3]) != 1
            @show "Something is not right here! - id:3"
        end
        l_u_index = 0;
        if cmp(vector_name, "l1") == 0
            global l1 = zeros(num_elements);
            l_u_index = 1;
        elseif cmp(vector_name, "u1") == 0
            global u1 = zeros(num_elements);
            l_u_index = 2;
        else
            @show "Something is not right here! - id:4"
        end
        t = t+1;
        IV_line = lines[t];
        if cmp(IV_line, "I,V") != 0
            @show "Something is not right here! - id:5"
        end
        t = t+1;
        t_last = get_info_end_line(lines, t, file_num_lines);
        if l_u_index == 1
            fill_vector(l1, num_elements, lines[t:t_last]);
        else
            fill_vector(u1, num_elements, lines[t:t_last]);
        end
        t = t_last+1;
    end
    return t
end

read_block (generic function with 1 method)

In [8]:
file_num_lines = length(lines)
m = 0;
n = 0;
line_counter = 1;
while line_counter < file_num_lines
    line_counter = read_block(line_counter);
end

### Get Intuition From Data:

In [44]:
function revise_resolution(m, res)
    num_elements = length(m);
    mp = copy(m);
    for i in 1:num_elements
        if abs(m[i]) < res
            mp[i] = 0;
        end
    end
    return mp
end

revise_resolution (generic function with 1 method)

In [45]:
l_tilde = revise_resolution(l1, 2e-5);
u_tilde = revise_resolution(u1, 2e-5);

In [46]:
function get_nonzero_indeces(l, u)
    nonzero_ind = zeros(Int64, 0);
    zero_indeces = zeros(Int64, 0);
    for i in 1:n
        if l[i]>0 || u[i]<0
            append!(nonzero_ind, i);
        else
            append!(zero_indeces, i);
        end
    end
    return nonzero_ind, zero_indeces
end

get_nonzero_indeces (generic function with 1 method)

In [47]:
nz_ind, z_ind = get_nonzero_indeces(l_tilde, u_tilde);

In [48]:
function get_trivial_indeces(l, u)
    trivial_indeces = zeros(Int64, 0);
    non_trivial_indeces = zeros(Int64, 0);
    for i in 1:n
        if l[i]==u[i]
            append!(trivial_indeces, i);
        else
            append!(non_trivial_indeces, i);
        end
    end
    return trivial_indeces, non_trivial_indeces
end

get_trivial_indeces (generic function with 1 method)

In [49]:
triv_ind, non_triv_ind = get_trivial_indeces(l1, u1);

In [50]:
function make_full_v0(vec_hat, non_trivial_ind, n_h)
    v0_full = zeros(n);
    for ind in 1:n_h
        i = non_trivial_ind[ind];
        v0_full[i] = vec_hat[ind];
    end
    return v0_full
end

make_full_v0 (generic function with 2 methods)

## Round 2 Question:

$$
minimize  \quad \left\lVert v \right\rVert_0 \\
s.t.: \quad Sv = 0 \\
 \qquad\qquad l^1 \leq v \leq u^1
$$

### Solving: Randomized Weighted Objective Method

In [51]:
eps = 1e-5;
p = 0.8;
NUM_RUNS = 20;

In [52]:
function W4(W, val0, is_rand)
    for i in 1:n_hat
        s = abs(val0[i]) + eps;
        W[i] = (1+s^p)/(s^(p+1));
        if is_rand
            W[i] = W[i] * rand()^3;
        end
    end
    return W
end

W4 (generic function with 1 method)

In [53]:
n_hat = length(non_triv_ind);
S_hat = S[:, non_triv_ind];
l1_hat = l1[non_triv_ind];
u1_hat = u1[non_triv_ind];

In [54]:
optimizer = GLPK.Optimizer()
v_hat = MOI.add_variables(optimizer, n_hat)
v_abs_hat = MOI.add_variables(optimizer, n_hat)
# ------------------------------------------------------------------------------------
for i in 1:n_hat
        MOI.add_constraint(optimizer, 
                           MOI.SingleVariable(v_hat[i]), 
                           MOI.Interval(l1_hat[i], u1_hat[i]))
end
# ------------------------------------------------------------------------------------
for (i,row) in enumerate(eachrow(S_hat))
        row_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(row, v_hat), 0.0);
        MOI.add_constraint(optimizer, row_function, MOI.EqualTo(0.0))
end
# ------------------------------------------------------------------------------------
for i in 1:n_hat
        abs_plus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, -1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
        abs_minus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, 1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
        MOI.add_constraint(optimizer, 
                           abs_plus, 
                           MOI.GreaterThan(0.0))
        MOI.add_constraint(optimizer, 
                           abs_minus, 
                           MOI.GreaterThan(0.0))
end

In [55]:
W_hat = ones(n_hat);   # Initial weights.
best_nnz = n_hat;
best_vec = ones(n_hat);

for k in 1:NUM_RUNS
    objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(W_hat, v_abs_hat), 0.0)
    MOI.set(optimizer, 
            MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
            objective_function)
    MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    
    MOI.optimize!(optimizer)
    status = MOI.get(optimizer, MOI.TerminationStatus())
    if status == MOI.OPTIMAL
        # Ok, we solved the problem!
        v0_hat = MOI.get(optimizer, MOI.VariablePrimal(), v_hat);
    else
        @show "Not Successful"
    end

    v0_hat = (abs.(v0_hat).>2e-5).*v0_hat;
    nnz = sum(abs.(v0_hat) .> 2e-5);
    @show nnz
    if nnz < best_nnz
        best_nnz = nnz;
        best_vec = v0_hat;
    end

    # Adjust the weights elementwise and re-iterate
    W_hat = W4(W_hat, v0_hat, true);
end

nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8
nnz = 8


In [56]:
best_v0 =  make_full_v0(best_vec, non_triv_ind, n_hat);

In [57]:
norm(best_v0, 0)

8.0

## Saving

In [ ]:
f = open("T1/output.txt", "w") do f
  for data_line in best_v0
    println(f, data_line)
  end
end